In [ ]:
!mkdir -p data
!mkdir -p embeddings
!mkdir -p embeddings/tf_idf
!mkdir -p embeddings/bert_nli
!mkdir -p embeddings/bert_pp
!rm -fr /content/data/sample_data
!rm -r data

In [ ]:
!pip install sentence_transformers
!pip install rank-bm25
!pip install simcse
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 14.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=5b0e0e8581865c04e4f39839845ae3d7366fbbf27dd25546894c5f637fd6a75d
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.o

In [ ]:
from modulefinder import packagePathMap
from unittest import result
from click import pass_obj
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi
import json
import jsonlines
import numpy as np
import math
import os
import argparse
import statistics
import codecs
import json
import csv

In [ ]:
model_sent_bert_nli = SentenceTransformer('nli-roberta-base-v2')

In [ ]:
model_sent_bert_pp = SentenceTransformer('paraphrase-TinyBERT-L6-v2')

In [ ]:
def get_bert_nli_embedding(sentence):
    return model_sent_bert_nli.encode(sentence)


def get_bert_pp_embedding(sentence):
    return model_sent_bert_pp.encode(sentence)


def tfidf_fit_corpus(corpus):
    tfidf = TfidfVectorizer().fit([corpus])
    return tfidf

def get_tfidf_embedding(tfidf, query):
    return tfidf.transform([query]).toarray()[0]

def get_bm25_score(corpus, query):
    tokenized_corpus = [doc.split(" ") for doc in corpus]
    tokenized_query = query.split(" ")
    bm25 = BM25Okapi(tokenized_corpus)
    return bm25.get_scores(tokenized_query)


In [ ]:
class Paper:
    def __init__(self, paper_id, metadata, title, abstract, pred_labels_truncated, pred_labels):
        self.paper_id = paper_id
        self.metadata = metadata
        self.title = title
        self.abstract = abstract
        self.pred_labels_truncated = pred_labels_truncated
        self.pred_labels = pred_labels

papers = dict()
with jsonlines.open('/content/abstracts-csfcube-preds.jsonl') as reader:
    for obj in reader:
        papers[obj['paper_id']] = Paper(obj['paper_id'], obj['metadata'], obj['title'], obj['abstract'], obj['pred_labels_truncated'], obj['pred_labels'])

f = open('/content/test-pid2anns-csfcube-background.json')
q_background = json.load(f)
q_background_ids = list(q_background.keys())

f = open('/content/test-pid2anns-csfcube-method.json')
q_method = json.load(f)
q_method_ids = list(q_method.keys())

f = open('/content/test-pid2anns-csfcube-result.json')
q_result = json.load(f)
q_result_ids = list(q_result.keys())

all_ids = list(papers.keys())

# candidate_ids = list(set(all_ids) - set(q_background_ids) - set(q_method_ids) - set(q_result_ids))

print(len(all_ids))
print(len(q_background_ids))
print(len(q_method_ids))
print(len(q_result_ids))
# print(len(candidate_ids))

def facet_sentences(pid, facet):
    labels = {'background': ['background_label', 'objective_label'], 'method': ['method_label'], 'result': ['result_label']}
    paper = papers[pid]
    s = list()
    for i in range(len(paper.pred_labels)):
        if paper.pred_labels[i] in labels[facet]:
            s.append(paper.abstract[i])
    return s

def merge_list_to_str(l):
    return " ".join(l)



4207
16
17
17


In [ ]:
corpus_list = list()
for pid in papers:
    doc = papers[pid].abstract
    corpus_list = corpus_list + doc
corpus = merge_list_to_str(corpus_list)
print(corpus_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
tfidf = tfidf_fit_corpus(corpus)

In [ ]:
# bert_nli all
d = dict()
err_pids = list()
i = 0
for pid in papers:
    # i += 1
    # if i % 50 == 0:
    #     print(i)
    try:
        doc = papers[pid].abstract
        if len(doc) == 0:
            d[pid] = list()
            continue
        bert_nli_array = np.array(get_bert_nli_embedding(merge_list_to_str(doc))).tolist()
        d[pid] = bert_nli_array
    except:
        err_pids.append(pid)
print(f'len(err_pids) = {len(err_pids)}')
print(err_pids)
with open("embeddings/bert_nli/all.json", "w") as outfile:
    json.dump(d, outfile)
d = json.load(open('embeddings/bert_nli/all.json'),
              parse_float=lambda x: round(float(x), 6))
with open('embeddings/bert_nli/all.json', "w") as outfile:
    json.dump(d, outfile)


len(err_pids) = 0
[]


FileNotFoundError: ignored

In [ ]:
# bert_nli queries
for i, ids in enumerate([q_background_ids, q_method_ids, q_result_ids]):
    d = dict()
    err_pids = list()
    s = ''
    if i == 0:
        s = 'background'
    elif i == 1:
        s = 'method'
    elif i == 2:
        s = 'result'
    for pid in ids:
        try:
            doc = facet_sentences(pid, s)
            if len(doc) == 0:
                d[pid] = list()
                continue
            bert_nli_array = np.array(get_bert_nli_embedding(merge_list_to_str(doc))).tolist()
            d[pid] = bert_nli_array
        except:
            err_pids.append(pid)
    print(f'len(err_pids), {s} = {len(err_pids)}')
    print(err_pids)
    with open(f'embeddings/bert_nli/{s}.json', "w") as outfile:
        json.dump(d, outfile)
    d = json.load(open(f'embeddings/bert_nli/{s}.json'),
                parse_float=lambda x: round(float(x), 6))
    with open(f'embeddings/bert_nli/{s}.json', "w") as outfile:
        json.dump(d, outfile)


len(err_pids), background = 0
[]
len(err_pids), method = 0
[]
len(err_pids), result = 0
[]


In [ ]:
# bert_pp all
d = dict()
err_pids = list()
i = 0
for pid in papers:
    # i += 1
    # if i % 50 == 0:
    #     print(i)
    try:
        doc = papers[pid].abstract
        if len(doc) == 0:
            d[pid] = list()
            continue
        bert_pp_array = np.array(get_bert_pp_embedding(merge_list_to_str(doc))).tolist()
        d[pid] = bert_pp_array
    except:
        err_pids.append(pid)
print(f'len(err_pids) = {len(err_pids)}')
print(err_pids)
with open("embeddings/bert_pp/all.json", "w") as outfile:
    json.dump(d, outfile)
d = json.load(open('embeddings/bert_pp/all.json'),
              parse_float=lambda x: round(float(x), 6))
with open('embeddings/bert_pp/all.json', "w") as outfile:
    json.dump(d, outfile)

len(err_pids) = 0
[]


In [ ]:
# bert_pp queries
for i, ids in enumerate([q_background_ids, q_method_ids, q_result_ids]):
    d = dict()
    err_pids = list()
    s = ''
    if i == 0:
        s = 'background'
    elif i == 1:
        s = 'method'
    elif i == 2:
        s = 'result'
    for pid in ids:
        try:
            doc = facet_sentences(pid, s)
            if len(doc) == 0:
                d[pid] = list()
                continue
            bert_pp_array = np.array(get_bert_pp_embedding(merge_list_to_str(doc))).tolist()
            d[pid] = bert_pp_array
        except:
            err_pids.append(pid)
    print(f'len(err_pids), {s} = {len(err_pids)}')
    print(err_pids)
    with open(f'embeddings/bert_pp/{s}.json', "w") as outfile:
        json.dump(d, outfile)
    d = json.load(open(f'embeddings/bert_pp/{s}.json'),
                parse_float=lambda x: round(float(x), 6))
    with open(f'embeddings/bert_pp/{s}.json', "w") as outfile:
        json.dump(d, outfile)

len(err_pids), background = 0
[]
len(err_pids), method = 0
[]
len(err_pids), result = 0
[]


In [ ]:
def mean_reciprocal_rank(rs):
    """Score is reciprocal of the rank of the first relevant item

    First element is 'rank 1'.  Relevance is binary (nonzero is relevant).

    Example from http://en.wikipedia.org/wiki/Mean_reciprocal_rank
    >>> rs = [[0, 0, 1], [0, 1, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.61111111111111105
    >>> rs = np.array([[0, 0, 0], [0, 1, 0], [1, 0, 0]])
    >>> mean_reciprocal_rank(rs)
    0.5
    >>> rs = [[0, 0, 0, 1], [1, 0, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.75

    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Mean reciprocal rank
    """
    rs = (np.asarray(r).nonzero()[0] for r in rs)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs])

In [ ]:
def r_precision_call(r):
    """Score is precision after all relevant documents have been retrieved

    Relevance is binary (nonzero is relevant).

    >>> r = [0, 0, 1]
    >>> r_precision(r)
    0.33333333333333331
    >>> r = [0, 1, 0]
    >>> r_precision(r)
    0.5
    >>> r = [1, 0, 0]
    >>> r_precision(r)
    1.0

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        R Precision
    """
    r = np.asarray(r) != 0
    z = r.nonzero()[0]
    if not z.size:
        return 0.
    return np.mean(r[:z[-1] + 1])

In [ ]:
def precision_at_k(r, k):
    """Score is precision @ k

    Relevance is binary (nonzero is relevant).

    >>> r = [0, 0, 1]
    >>> precision_at_k(r, 1)
    0.0
    >>> precision_at_k(r, 2)
    0.0
    >>> precision_at_k(r, 3)
    0.33333333333333331
    >>> precision_at_k(r, 4)
    Traceback (most recent call last):
        File "<stdin>", line 1, in ?
    ValueError: Relevance score length < k


    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Precision @ k

    Raises:
        ValueError: len(r) must be >= k
    """
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    return np.mean(r)

In [ ]:
def average_precision(r):
    """Score is average precision (area under PR curve)

    Relevance is binary (nonzero is relevant).

    >>> r = [1, 1, 0, 1, 0, 1, 0, 0, 0, 1]
    >>> delta_r = 1. / sum(r)
    >>> sum([sum(r[:x + 1]) / (x + 1.) * delta_r for x, y in enumerate(r) if y])
    0.7833333333333333
    >>> average_precision(r)
    0.78333333333333333

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Average precision
    """
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

In [ ]:
def mean_average_precision(rs):
    """Score is mean average precision

    Relevance is binary (nonzero is relevant).

    >>> rs = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1]]
    >>> mean_average_precision(rs)
    0.78333333333333333
    >>> rs = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1], [0]]
    >>> mean_average_precision(rs)
    0.39166666666666666

    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Mean average precision
    """
    return np.mean([average_precision(r) for r in rs])

In [ ]:
def dcg_at_k(r, k, method=1):
    """Score is discounted cumulative gain (dcg)

    Relevance is positive real values.  Can use binary
    as the previous methods.

    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> dcg_at_k(r, 1)
    3.0
    >>> dcg_at_k(r, 1, method=1)
    3.0
    >>> dcg_at_k(r, 2)
    5.0
    >>> dcg_at_k(r, 2, method=1)
    4.2618595071429155
    >>> dcg_at_k(r, 10)
    9.6051177391888114
    >>> dcg_at_k(r, 11)
    9.6051177391888114
    "An Introduction to Neural Information Retrieval" writes method 1 for DCG
    so using that as default.
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]

    Returns:
        Discounted cumulative gain
    """
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0

In [ ]:
def ndcg_at_k(r, k, method=0):
    """Score is normalized discounted cumulative gain (ndcg)

    Relevance is positive real values.  Can use binary
    as the previous methods.

    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> ndcg_at_k(r, 1)
    1.0
    >>> r = [2, 1, 2, 0]
    >>> ndcg_at_k(r, 4)
    0.9203032077642922
    >>> ndcg_at_k(r, 4, method=1)
    0.96519546960144276
    >>> ndcg_at_k([0], 1)
    0.0
    >>> ndcg_at_k([1], 2)
    1.0

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]

    Returns:
        Normalized discounted cumulative gain
    """
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [ ]:
# paper-ids for the different folds for cross-validation; Ugly but easier to have it here.
facet2folds = {
    "background": {"fold1_dev": ["3264891_background", "1936997_background", "11844559_background",
                                 "52194540_background", "1791179_background", "6431039_background",
                                 "6173686_background", "7898033_background"],
                   "fold2_dev": ["5764728_background", "10014168_background", "10695055_background",
                                 "929877_background", "1587_background", "51977123_background",
                                 "8781666_background", "189897839_background"],
                   "fold1_test": ["5764728_background", "10014168_background", "10695055_background",
                                  "929877_background", "1587_background", "51977123_background",
                                  "8781666_background", "189897839_background"],
                   "fold2_test": ["3264891_background", "1936997_background", "11844559_background",
                                  "52194540_background", "1791179_background", "6431039_background",
                                  "6173686_background", "7898033_background"]},
    "method": {"fold1_dev": ["189897839_method", "1791179_method", "11310392_method", "2468783_method",
                             "13949438_method", "5270848_method", "52194540_method", "929877_method"],
               "fold2_dev": ["5052952_method", "10010426_method", "102353905_method", "174799296_method",
                             "1198964_method", "53080736_method", "1936997_method", "80628431_method",
                             "53082542_method"],
               "fold1_test": ["5052952_method", "10010426_method", "102353905_method", "174799296_method",
                              "1198964_method", "53080736_method", "1936997_method", "80628431_method",
                              "53082542_method"],
               "fold2_test": ["189897839_method", "1791179_method", "11310392_method", "2468783_method",
                              "13949438_method", "5270848_method", "52194540_method", "929877_method"]},
    "result": {"fold1_dev": ["2090262_result", "174799296_result", "11844559_result", "2468783_result",
                             "1306065_result", "5052952_result", "3264891_result", "8781666_result"],
               "fold2_dev": ["2865563_result", "10052042_result", "11629674_result", "1587_result",
                             "1198964_result", "53080736_result", "2360770_result", "80628431_result",
                             "6431039_result"],
               "fold1_test": ["2865563_result", "10052042_result", "11629674_result", "1587_result",
                              "1198964_result", "53080736_result", "2360770_result", "80628431_result",
                              "6431039_result"],
               "fold2_test": ["2090262_result", "174799296_result", "11844559_result", "2468783_result",
                              "1306065_result", "5052952_result", "3264891_result", "8781666_result"]},
    "all": {"fold1_dev": ["3264891_background", "1936997_background", "11844559_background",
                          "52194540_background", "1791179_background", "6431039_background",
                          "6173686_background", "7898033_background", "189897839_method",
                          "1791179_method", "11310392_method", "2468783_method", "13949438_method",
                          "5270848_method", "52194540_method", "929877_method", "2090262_result",
                          "174799296_result", "11844559_result", "2468783_result", "1306065_result",
                          "5052952_result", "3264891_result", "8781666_result"],
            "fold2_dev": ["5764728_background", "10014168_background", "10695055_background",
                          "929877_background", "1587_background", "51977123_background",
                          "8781666_background", "189897839_background", "5052952_method", "10010426_method",
                          "102353905_method", "174799296_method", "1198964_method", "53080736_method",
                          "1936997_method", "80628431_method", "53082542_method", "2865563_result",
                          "10052042_result", "11629674_result", "1587_result", "1198964_result",
                          "53080736_result", "2360770_result", "80628431_result", "6431039_result"],
            "fold1_test": ["5764728_background", "10014168_background", "10695055_background",
                           "929877_background", "1587_background", "51977123_background", "8781666_background",
                           "189897839_background", "5052952_method", "10010426_method", "102353905_method",
                           "174799296_method", "1198964_method", "53080736_method", "1936997_method",
                           "80628431_method", "53082542_method", "2865563_result", "10052042_result",
                           "11629674_result", "1587_result", "1198964_result", "53080736_result",
                           "2360770_result", "80628431_result", "6431039_result"],
            "fold2_test": ["3264891_background", "1936997_background", "11844559_background",
                           "52194540_background", "1791179_background", "6431039_background",
                           "6173686_background", "7898033_background", "189897839_method", "1791179_method",
                           "11310392_method", "2468783_method", "13949438_method", "5270848_method",
                           "52194540_method", "929877_method", "2090262_result", "174799296_result",
                           "11844559_result", "2468783_result", "1306065_result", "5052952_result",
                           "3264891_result", "8781666_result"]
            }
}

In [ ]:
def recall_at_k(ranked_rel, atk, max_total_relevant):
    """
    Compute recall at k.
    :param ranked_rel: list(int); ranked list of relevance judged data.
    :param atk: int; rank at which to compute metric.
    :param max_total_relevant: int; maximum relevant to consider in
        case there are more relevant in total.
    :return: recall: float.
    """
    total_relevant = sum(ranked_rel)
    total_relevant = min(max_total_relevant, total_relevant)
    relatk = sum(ranked_rel[:atk])
    if total_relevant > 0:
        recall_atk = float(relatk)/total_relevant
    else:
        recall_atk = 0.0
    return recall_atk

In [ ]:
def read_facet_specific_relevances(data_path, run_path, dataset, facet, method_name):
    """
    Read the gold data and the model rankings and the relevances for the
    model.
    :param data_path: string; directory with gold citations for test pids and rankings
        from baseline methods in subdirectories.
    :param run_path: string; directory with ranked candidates for baselines a subdir of
        data_path else is a model run.
    :param method_name: string; method with which ranks were created.
    :param dataset: string; eval dataset.
    :param facet: string; facet for eval.
    :return: qpid2rankedcand_relevances: dict('qpid_facet': [relevances]);
        candidate gold relevances for the candidates in order ranked by the
        model.
    """
    gold_fname = os.path.join(data_path, 'test-pid2anns-{:s}-{:s}.json'.format(dataset, facet))
    ranked_fname = os.path.join(run_path, 'test-pid2pool-{:s}-{:s}-{:s}-ranked.json'.
                                format(dataset, method_name, facet))
    # Load gold test data (citations).
    with codecs.open(gold_fname, 'r', 'utf-8') as fp:
        pid2pool_source = json.load(fp)
        num_query = len(pid2pool_source)
        print('Gold query pids: {:d}'.format(num_query))
        pid2rels_gold = {}
        for qpid, pool_rel in pid2pool_source.items():
            pool = pool_rel['cands']
            cands_rels = pool_rel['relevance_adju']
            pid2rels_gold['{:s}_{:s}'.format(qpid, facet)] = \
                dict([(pid, rel) for pid, rel in zip(pool, cands_rels)])
    # Load ranked predictions on test data with methods.
    with codecs.open(ranked_fname, 'r', 'utf-8') as fp:
        pid2ranks = json.load(fp)
        print('Valid ranked query pids: {:d}'.format(len(pid2ranks)))
        qpid2rankedcand_relevances = {}
        for qpid, citranks in pid2ranks.items():
            candpids = [pid_score[0] for pid_score in citranks if isinstance(pid_score, list)]
            cand_relevances = [pid2rels_gold['{:s}_{:s}'.format(qpid, facet)][pid] for pid in candpids]
            qpid2rankedcand_relevances['{:s}_{:s}'.format(qpid, facet)] = cand_relevances
    return qpid2rankedcand_relevances


In [ ]:
def read_all_facet_relevances(data_path, run_path, dataset, method_name, facets):
    """
    Read the gold data and the model rankings and the relevances for the
    model.
    :param data_path: string; directory with gold citations for test pids and rankings
        from baseline methods in subdirectories.
    :param run_path: string; directory with ranked candidates for baselines a subdir of
        data_path else is a model run.
    :param method_name: string; method with which ranks were created.
    :param dataset: string; eval dataset.
    :param facets: list(string); what facets to read/what counts as "all".
    :return: qpid2rankedcand_relevances: dict('qpid_facet': [relevances]);
        candidate gold relevances for the candidates in order ranked by the
        model.
    """
    qpid2rankedcand_relevances = {}
    for facet in facets:
        print('Reading facet: {:s}'.format(facet))
        gold_fname = os.path.join(data_path, 'test-pid2anns-{:s}-{:s}.json'.format(dataset, facet))
        ranked_fname = os.path.join(run_path, 'test-pid2pool-{:s}-{:s}-{:s}-ranked.json'.
                                    format(dataset, method_name, facet))
        # Load gold test data (citations).
        with codecs.open(gold_fname, 'r', 'utf-8') as fp:
            pid2pool_source = json.load(fp)
            num_query = len(pid2pool_source)
            print('Gold query pids: {:d}'.format(num_query))
            pid2rels_gold = {}
            for qpid, pool_rel in pid2pool_source.items():
                pool = pool_rel['cands']
                cands_rels = pool_rel['relevance_adju']
                pid2rels_gold['{:s}_{:s}'.format(qpid, facet)] = \
                    dict([(pid, rel) for pid, rel in zip(pool, cands_rels)])
        # Load ranked predictions on test data with methods.
        with codecs.open(ranked_fname, 'r', 'utf-8') as fp:
            pid2ranks = json.load(fp)
            print('Valid ranked query pids: {:d}'.format(len(pid2ranks)))
            for qpid, citranks in pid2ranks.items():
                candpids = [pid_score[0] for pid_score in citranks if isinstance(pid_score, list)]
                cand_relevances = [pid2rels_gold['{:s}_{:s}'.format(qpid, facet)][pid] for pid in candpids]
                qpid2rankedcand_relevances['{:s}_{:s}'.format(qpid, facet)] = cand_relevances
    print('Total queries: {:d}'.format(len(qpid2rankedcand_relevances)))
    return qpid2rankedcand_relevances


In [ ]:
def compute_metrics(ranked_judgements, pr_atk, threshold_grade):
    """
    Given the ranked judgements compute the metrics for a query.
    :param ranked_judgements: list(int); graded or binary relevances in rank order.
    :param pr_atk: int; the @K value to use for computing precision and recall.
    :param threshold_grade: int; Assuming 0-3 graded relevances, threshold at some point
        and convert graded to binary relevance.
    :return:
    """
    graded_judgements = ranked_judgements
    ranked_judgements = [1 if rel >= threshold_grade else 0 for rel in graded_judgements]
    # Use the full set of candidate not the pr_atk.
    ndcg = ndcg_at_k(graded_judgements, len(ranked_judgements))
    ndcg_pr = ndcg_at_k(graded_judgements, int(0.20*len(ranked_judgements)))
    ndcg_20 = ndcg_at_k(graded_judgements, 20)
    max_total_relevant = sum(ranked_judgements)
    recall = recall_at_k(ranked_rel=ranked_judgements,
                         atk=pr_atk, max_total_relevant=max_total_relevant)
    precision = precision_at_k(r=ranked_judgements, k=pr_atk)
    r_precision = r_precision_call(r=ranked_judgements)
    f1 = 2*precision*recall/(precision + recall) if (precision + recall) > 0 else 0.0
    av_precision = average_precision(r=ranked_judgements)
    reciprocal_rank = mean_reciprocal_rank(rs=[ranked_judgements])
    metrics = {
        'recall': float(recall),
        'precision': float(precision),
        'f1': float(f1),
        'r_precision': float(r_precision),
        'av_precision': float(av_precision),
        'reciprocal_rank': float(reciprocal_rank),
        'ndcg': ndcg,
        'ndcg@20': ndcg_20,
        'ndcg%20': ndcg_pr
    }
    return metrics

In [ ]:
def aggregate_metrics_crossval(query_metrics, split_str, facet_str):
    """
    Given metrics over individual queries aggregate over different
    queries.
    :param query_metrics: dict(query_id: metrics_dict from compute_metrics)
    :param split_str: string; {dev, test}
    :param facet_str: string; {background, method, result}
    :return:
    """
    aggmetrics = {
        'precision': [],
        'recall': [],
        'f1': [],
        'r_precision': [],
        'mean_av_precision': [],
        'mean_reciprocal_rank': [],
        'ndcg': [],
        'ndcg@20': [],
        'ndcg%20': []
    }
    # For dev only use a part of the fold - using both makes it identical to test.
    if split_str == 'dev':
        folds = ['fold1_{:s}'.format(split_str)]
    elif split_str == 'test':
        folds = ['fold1_{:s}'.format(split_str), 'fold2_{:s}'.format(split_str)]
    for fold_str in folds:
        fold_pids = facet2folds[facet_str][fold_str]
        precision, recall, f1, av_precision, mrr, ndcg, r_precision = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
        ndcg_20, ndcg_pr = 0.0, 0.0
        for query_id in fold_pids:
            # Aggregate across paper types in the fold.
            metrics = query_metrics[query_id]
            # Aggregate across all papers in the fold
            precision += metrics['precision']
            recall += metrics['recall']
            f1 += metrics['f1']
            av_precision += metrics['av_precision']
            mrr += metrics['reciprocal_rank']
            r_precision += metrics['r_precision']
            ndcg += metrics['ndcg']
            ndcg_20 += metrics['ndcg@20']
            ndcg_pr += metrics['ndcg%20']
        # Average all folds
        num_queries = len(fold_pids)
        precision, recall, f1 = precision/num_queries, recall/num_queries, f1/num_queries
        av_precision = av_precision/num_queries
        mrr, ndcg, r_precision = mrr/num_queries, ndcg/num_queries, r_precision/num_queries
        ndcg_20, ndcg_pr = ndcg_20/num_queries, ndcg_pr/num_queries
        # Save the averaged metric for every fold.
        aggmetrics['precision'].append(precision)
        aggmetrics['recall'].append(recall)
        aggmetrics['f1'].append(f1)
        aggmetrics['r_precision'].append(r_precision)
        aggmetrics['mean_av_precision'].append(av_precision)
        aggmetrics['mean_reciprocal_rank'].append(mrr)
        aggmetrics['ndcg'].append(ndcg)
        aggmetrics['ndcg@20'].append(ndcg_20)
        aggmetrics['ndcg%20'].append(ndcg_pr)

    aggmetrics = {
        'precision': statistics.mean(aggmetrics['precision']),
        'recall': statistics.mean(aggmetrics['recall']),
        'f1': statistics.mean(aggmetrics['f1']),
        'r_precision': statistics.mean(aggmetrics['r_precision']),
        'mean_av_precision': statistics.mean(aggmetrics['mean_av_precision']),
        'mean_reciprocal_rank': statistics.mean(aggmetrics['mean_reciprocal_rank']),
        'ndcg': statistics.mean(aggmetrics['ndcg']),
        'ndcg@20': statistics.mean(aggmetrics['ndcg@20']),
        'ndcg%20': statistics.mean(aggmetrics['ndcg%20'])
    }
    return aggmetrics

In [ ]:
def graded_eval_pool_rerank(data_path, run_path, method_name, dataset, facet, split):
    """
    Evaluate the re-ranked pool for the faceted data. Anns use graded relevance scores.
    :param data_path: string; directory with gold citations for test pids and rankings
        from baseline methods in subdirectories.
    :param run_path: string; directory with ranked candidates for baselines a subdir of
        data_path else is a model run.
    :param method_name: string; method with which ranks were created.
    :param dataset: string; eval dataset.
    :param facet: string; facet for eval.
    :param split: strong; {dev, test}
    :return:
    """
    print(f'EVAL SPLIT: {split}')
    ATK = 20
    with codecs.open(os.path.join(data_path, 'queries-release.csv')) as csvfile:
        reader = csv.DictReader(csvfile)
        query_metadata = dict([('{:s}_{:s}'.format(row['pid'], row['facet']), row) for row in reader])

    perq_out_fname = os.path.join(run_path, 'test-pid2pool-{:s}-{:s}-{:s}-perq.csv'.
                                  format(dataset, method_name, facet))
    if facet == 'all':
        qpid2rankedcand_relevances = read_all_facet_relevances(data_path=data_path, run_path=run_path,
                                                               dataset=dataset, method_name=method_name,
                                                               facets=['background', 'method', 'result'])
    else:
        qpid2rankedcand_relevances = read_facet_specific_relevances(data_path=data_path, run_path=run_path,
                                                                    dataset=dataset, facet=facet,
                                                                    method_name=method_name)
    # Go over test papers and compute metrics.
    all_metrics = {}
    num_cands = 0.0
    num_queries = 0.0
    perq_file = codecs.open(perq_out_fname, 'w', 'utf-8')
    perq_csv = csv.DictWriter(perq_file, extrasaction='ignore',
                              fieldnames=['paper_id', 'title', 'recall', 'precision', 'f1', 'r_precision',
                                          'av_precision', 'reciprocal_rank', 'ndcg', 'ndcg%20', 'paper type'])
    perq_csv.writeheader()
    print('Precision and recall at rank: {:d}'.format(ATK))
    for qpid_facet, qranked_judgements in qpid2rankedcand_relevances.items():
        all_metrics[qpid_facet] = compute_metrics(qranked_judgements, pr_atk=ATK,
                                                  threshold_grade=2)
        num_cands += len(qranked_judgements)
        num_queries += 1
        metrics = all_metrics[qpid_facet]
        metrics['paper_id'] = qpid_facet
        metrics['title'] = query_metadata[qpid_facet]['title']
        metrics['paper type'] = query_metadata[qpid_facet]['paper type']
        metrics['year'] = query_metadata[qpid_facet]['year']
        perq_csv.writerow(metrics)
    aggmetrics = aggregate_metrics_crossval(query_metrics=all_metrics,
                                            facet_str=facet, split_str=split)
    print('Wrote: {:s}'.format(perq_file.name))
    perq_file.close()
    print('Total queries: {:d}; Total candidates: {:d}'.format(int(num_queries), int(num_cands)))
    print('R-Precision; Precision@{:d}; Recall@{:d}; NDCG; NDCG@20; NDCG%20'.format(ATK, ATK))
    print('{:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}\n'.
          format(aggmetrics['r_precision'], aggmetrics['precision'], aggmetrics['recall'],
                 aggmetrics['ndcg'], aggmetrics['ndcg@20'], aggmetrics['ndcg%20']))

In [ ]:
graded_eval_pool_rerank(data_path="/content/", method_name="specter",
                        facet="background", dataset='csfcube', run_path="/content/embeddings/tf_idf",
                        split='dev')
graded_eval_pool_rerank(data_path="/content/", method_name="specter",
                        facet="background", dataset='csfcube', run_path="/content/embeddings/tf_idf",
                        split='test')

EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: /content/embeddings/tf_idf/test-pid2pool-csfcube-specter-background-perq.csv
Total queries: 16; Total candidates: 0
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0000, nan, 0.0000, 0.0000, 0.0000, 0.0000

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: /content/embeddings/tf_idf/test-pid2pool-csfcube-specter-background-perq.csv
Total queries: 16; Total candidates: 0
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0000, nan, 0.0000, 0.0000, 0.0000, 0.0000



/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  >>> a[1, :] = 0.1
/usr/local/lib/python3.9/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  # Compute the mean.


In [ ]:
!pip install rank_bm25
from rank_bm25 import BM25Okapi
import numpy as np
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import preprocess_documents, preprocess_string

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
bm25_index = BM25Okapi(corpus)

In [ ]:
def search(search_string, num_results=len(corpus)):
    search_tokens = preprocess_string(search_string)
    scores = bm25_index.get_scores(search_tokens)
    top_indexes = np.argsort(scores)[::-1][:num_results]
    for i in range(0, 10):
      print(scores[i])
    return top_indexes, scores

In [ ]:
indexes, scores = search('novel coronavirus treatment')

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [ ]:
indexes

array([4519045, 1506350, 1506344, ..., 3012689, 3012688,       0])

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 89.3 MB/s eta 0:00:00


In [3]:
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pickle
import os
from transformers import BertTokenizer

In [4]:
class MNLIDataBert(Dataset):

  def __init__(self, train_df, val_df):
    self.label_dict = {'entailment': 0, 'contradiction': 1, 'neutral': 2}

    self.train_df = train_df
    self.val_df = val_df

    self.base_path = '/content/'
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) # Using a pre-trained BERT tokenizer to encode sentences
    self.train_data = None
    self.val_data = None
    self.init_data()

  def init_data(self):
    self.train_data = self.load_data(self.train_df)
    self.val_data = self.load_data(self.val_df)

  def load_data(self, df):
    MAX_LEN = 512
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []

    premise_list = df['sentence1'].to_list()
    hypothesis_list = df['sentence2'].to_list()
    label_list = df['gold_label'].to_list()

    for (premise, hypothesis, label) in zip(premise_list, hypothesis_list, label_list):
      premise_id = self.tokenizer.encode(premise, add_special_tokens = False)
      hypothesis_id = self.tokenizer.encode(hypothesis, add_special_tokens = False)
      pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]
      premise_len = len(premise_id)
      hypothesis_len = len(hypothesis_id)

      segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))  # sentence 0 and sentence 1
      attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))  # mask padded values

      token_ids.append(torch.tensor(pair_token_ids))
      seg_ids.append(segment_ids)
      mask_ids.append(attention_mask_ids)
      y.append(self.label_dict[label])
    
    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    seg_ids = pad_sequence(seg_ids, batch_first=True)
    y = torch.tensor(y)
    dataset = TensorDataset(token_ids, mask_ids, seg_ids, y)
    print(len(dataset))
    return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
    train_loader = DataLoader(
      self.train_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    val_loader = DataLoader(
      self.val_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    return train_loader, val_loader
  
mnli_dataset = MNLIDataBert(train_df, val_df)

NameError: ignored